# MCB 32: Lab 04 - Muscle Excitability and Contractility
 
---

### Professor Robin Ball

During this lab, you will get more practice using Python and creating graphs.

*Estimated Time: ~1 Hour*

---

### Table of Contents

1. [Entering your data](#section-1)<br>
2. [Graphing your data](#section-2)<br>

In [1]:
#Run this cell first! Everytime you come back to this notebook, you need to run this cell again.

from datascience import *
import pandas as pd
import numpy as np
import qgrid
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
grid_options = {"forceFitColumns" : False, "defaultColumnWidth" : 175}

## 1. Entering your data <a id='section-1'></a>
Before you can graph your data, you must enter it into the computer. The cell below will load a spreadsheet which is already filled in with the sample data for Table 1.
</div>

In [ ]:
table_1 = Table().read_table("lab4-table-1.csv")
table_1

The cell below will load a spreadsheet which is already filled in with the sample data for Table 4.
</div>

In [ ]:
table_4 = Table().read_table("lab4-table-4.csv")
table_4

## 2. Graphing your data <a id='section-2'></a>

We're interested in a function called `plot`, which makes line graphs. We can tell the computer to find `plot` in the `plt` module using what's called *dot notation*:

In [ ]:
# use the plot function
plt.plot()

`plot` needs to get data to plot as its arguments. In this case, we can give it the lists of data we made earlier. The data goes inside the parentheses: the data that goes on the x-axis is the first argument, and the data that goes on the y-axis is the second.

Here's an example of how to plot Stimulus Amplitude against Twitch Duration for Subject 1. Stimulus Amplitude is on the x-axis, so it goes first in the parentheses.

In [ ]:
# create a line plot of data for subject 1
plt.plot(table_1["Stimulus Amplitude 1 (mA)"], table_1["Twitch Duration 1 (ms)"])

# a function to show the line plot
plt.show()

Note how we accessed the values in our table as this will be important later on. To access a column of the table, use the bracket notation (known as "slicing"):

```python
table["Column Name"]
```

The cell below will access the stimulus amplitude for subject 2:

In [ ]:
table_1["Stimulus Amplitude 2 (mA)"]

As good scientists, we know that this graph is missing a lot. Our axes, graph, and line should all be labeled. Fortunately, the `plt` module has functions to do all of those things. 

Read over the call expressions in the next cell to figure out what each does, then run the cell to make the graph.

<div class="alert alert-warning">

Note that we put labels and titles in quotation marks. This tells the computer that they are text data (also called *strings*). If we didn't put them in quotes, the computer would think they are variables, then throw a fit when it couldn't find what they stood for.

</div>

In [ ]:
# create the line plot for subject 1
plt.plot(table_1["Stimulus Amplitude 1 (mA)"], table_1["Twitch Duration 1 (ms)"], label="Subject 1")

# label the x-axis
plt.xlabel("Stimulus amplitude (mA)")

# label the y-axis
plt.ylabel("Twitch duration (ms)")

# title the graph
plt.title("Twitch duration vs. stimulus strength")

# show the legend
plt.legend()

# show the plot
plt.show()

Now it's time to create your graph!

<div class='alert alert-info'>
    
**QUESTION:** Using the functions you learned in lab 0 and the refresher above, create a line plot of your data. Make one graph from Table 1 of twitch tension vs stimulus amplitude that includes lines for both subjects. Then make another line plot from Table 4 of twitch tension vs the frequency of stimulation.

</div>

In [ ]:
#Graph of table 1, twitch tension vs stimulus amplitude for both subjects 
#Hint: you will need to run the plt.plot function twice, once for each subject

...

In [ ]:
#Graph of table 4, twitch tension vs the frequency of stimulation
...

<div class='alert alert-warning'>
If you are not getting a nice trend line, check that your data has been entered in order of increasing frequency.
</div>

### Saving the Notebook as a PDF

Congrats on finishing your first lab notebook! To turn in this lab assignment follow the steps below:

>1. Press `Control + P` (or `Command + P` on Mac) to open the Print preview
2. Change the destination so that it saves locally on your own computer.
3. Save as PDF
4. If you are stuck, follow further instructions [here](https://www.wikihow.com/Save-a-Web-Page-as-a-PDF-in-Google-Chrome).
5. Upload your complete notebook in the lab 4 assignment on bCourses.

---
Notebook developed by: Monica Wilkinson and Chris Pyles

Data Science Modules: http://data.berkeley.edu/education/modules
